In [1]:
import pandas as pd
import owlready2 as owlr 
#https://pythonhosted.org/Owlready/
import rdflib
from rdflib import URIRef, Literal
from rdflib.namespace import RDF, RDFS
from rdflib import Namespace

from collections import Counter
import uuid

import io
import pydotplus
from IPython.display import display, Image
from rdflib.tools.rdf2dot import rdf2dot
from IPython.core.display import HTML

def visualize(g):
    stream = io.StringIO()
    rdf2dot(g, stream, opts = {display})
    dg = pydotplus.graph_from_dot_data(stream.getvalue())
    png = dg.create_png()

    display(Image(png))


import networkx as nx
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
raw_world = owlr.World(filename='raw_world.sqlite3')
rawdms = raw_world.get_ontology("datamodels_rdf.owl").load()
DMNS = Namespace("datamodels_rdf.owl")
onto = rawdms

OperationalError: database is locked

In [ ]:
DMNS.DataModel

In [ ]:
onto_dataproperties = list(onto.data_properties())
onto_individuals = list (onto.individuals())
individuals = [(i, i.is_a.first()) for i in onto_individuals + onto_dataproperties]
serializations = [i[0] for i in individuals if i[1] == onto.Serialization]

for s in serializations:
    print(s)
    mappings = [m for m in s.SerializationContainsMapping] 
    for m in mappings:
        print ("\t", m, m.is_a)

In [ ]:
with onto:
    owlr.sync_reasoner(raw_world)

In [ ]:
mapped_things = [(c,c.MappingKey,"class") for c in onto.classes() if len(c.MappingKey) > 0] + \
                [(p, p.MappingKey,"data") for p in onto.data_properties() if len(p.MappingKey) > 0] + \
                [(p, p.MappingKey,"object") for p in onto.object_properties() if len(p.MappingKey) > 0] 
mappable_things = {vv:k for k,v,t in mapped_things for vv in v}
mappable_things_ext = {vv:(k,t) for k,v,t in mapped_things for vv in v}
assert all([v==1 for v in Counter([b for c in mapped_things for b in c[1]]).values()])
#del mapped_things
#mappable_things
mappable_things_ext

row_node_class = [c for c in onto.classes() if c.name == 'RowNode']
row_node_participant = [p for p in onto.properties() if p.name == 'RowNodeParticipant']
row_node_class, row_node_participant
mappable_things_ext

In [ ]:
mapped_things

In [ ]:
mapping_data = {}
for s in serializations:
    mapping_data[s.name]={}
    mappings = [m for m in s.SerializationContainsMapping] 
    for m in mappings:
        mapping_data[s.name][m.name]={}
        for p in m.get_properties():
            mapping_data[s.name][m.name][p.name]=p[m].first()
    

In [ ]:
mapping_data['DomainModelClassEntityRecord']['AttributeMapping']

In [ ]:
data_rows = [{ "Business Domain" : "Core", 
             "Model" : "Principal Interactions", 
             "ModelType" : "Conceptual", 
             "Entity" : "Person", 
             "Attribute" : "PersonID", 
             "DataType" : "string"}]

In [ ]:
    

def gen_temp_entity_triples_from_row(s_mappings, onto_map, namespace, row, row_id):

    object_triples=[]
    row_node_uri = URIRef(uuid.uuid4().hex, namespace)
    rne = URIRef(onto_map['isRowNode'][0].iri)
    rnp = URIRef(onto_map['isParticipant'][0].iri)
    
    row_node_triple = (row_node_uri, RDF.type, rne)
    row_node_identifier_p = URIRef(onto_map['UniqueIdentifier'][0].iri)
    row_node_identifier_triple = (row_node_uri, row_node_identifier_p, Literal(row_id))
    
    for m_key, m_d in mapping_data['DomainModelClassEntityRecord'].items():
        #print(m_key, m_d)
        if m_d['MappingSubType']=="Entity":
#            print("Create Temporary Entity")
            if m_d['SerializationLabel'] in row:
                data = row[m_d['SerializationLabel']]
                e_type = onto_map[m_d['MappingLabel']]
#                print(e_type[0], ":", data)
                key = uuid.uuid4().hex
                thing_URI = URIRef(key, namespace)
                # RDF:type, RDFS:label
                props = { RDF.type : URIRef(e_type[0].iri), 
                          RDFS.label : Literal(data)}

                for k,v in props.items():
                    object_triples.append((thing_URI, k, v))
#                print()

    object_triples.append(row_node_triple)



# Create a graph containing the newly created object_triples
    row_g = rdflib.Graph()
    
    for triple in object_triples:
#        print(triple)
        row_g.add(triple)
        
    property_triples=[]
    property_triples.append(row_node_identifier_triple)
    
    # Assign all objects downstream of RowNodeParticipant 
    for s,p,o in object_triples:
        property_triples.append((row_node_uri, rnp, s))
        
    
    for m_key, m_d in mapping_data['DomainModelClassEntityRecord'].items():
        #print(m_key, m_d)
        if m_d['MappingSubType']=="Property":
#            print("Create Property Link")
            if m_d['SerializationLabel'] in row:
                data = row[m_d['SerializationLabel']] # Get the value in the row - normally the target object label
                prop = onto_map[m_d['MappingLabel']][0] # Get the property being referenced
                prop_domain = onto_map[m_d['MappingDomain']][0] # Get the domain class (used for search in the graph)
                prop_range = onto_map[m_d['MappingRange']][0] # Get the range class (used for search)
                prop_type = onto_map[m_d['MappingRange']][1]
#                print("!", prop, prop_domain, prop_range, prop_type)
                prop_iri = URIRef(prop.iri)
                subj_node = get_thing_from_label_graphsearch(row_g, prop_domain, search_key=None)[0]
                if prop_type == "class":
                    obj_node = get_thing_from_label_graphsearch(row_g, prop_range, search_key=data)[0]
                elif prop_type == "data" : 
                    obj_node = Literal(data)
#                print ("triple", subj_node, prop_iri, obj_node)
                property_triples.append((subj_node, prop_iri, obj_node))
    
    

    for triple in property_triples:
#        print(triple)
        row_g.add(triple)
    
    # Now there exist object_triples, property_triples
    # Additionally, we want some extra ontology driven triples for uniqueidentifiers 
    # build from the row-level class hierarchy
    
    
    return row_g


def t2rdflibg(triples): #triples to rdflib graph
    g = rdflib.Graph()
    for t in triples:
        g.add(t)
    return g

def get_key_values(g, key=RDFS.label, invert=False):
    if not invert:
        return list([(s, o.value) for s,p,o in g.triples((None, key, None))])
    else:
        return list([(o.value,s) for s,p,o in g.triples((None, key, None))])
    
    

def get_thing_from_label_graphsearch(graph, domain, search_key=None):
    #sd_list = [c for c in list(space.classes()) if c==domain]
    
    sd_list = [s for s,p,o in graph.triples((None, RDF.type, URIRef(domain.iri)))]
#    print("sd_list:", sd_list)
    # Get instances of classes that match provided keys
    if search_key is None:
        c_s = [s for s in sd_list]
    else:
        c_s = [s for l in sd_list for s,p,o in graph.triples((None, RDFS.label, Literal(search_key)))]
    
    return c_s

def text_process(text):
    return text.lower().replace(" ", "_")

# Traverse a graph g, from some source s and generate a list of all precursor items.
def nx_walk_predecessors(g,s,l=None,invert=False,process=None):
    if l is None:
        l=[]
    nxt=list(g.predecessors(s))

    if process is None:
        p_s=s
    else:
        p_s=process(s)
        
    if len(nxt)>0:

        l.extend([p_s])            

        for n in nxt:

            nx_walk_predecessors(g,n,l,process=process)
    else:
        l.extend([p_s])     
    if invert:
        return l[::-1]
    else:
        return l

# Prepare Unique Identity Annotation Triples 
# Get container tree
def gen_unique_identity_triples(graph, namespace_d):
    row_hierarchy_sparql="""SELECT distinct ?sl ?ol
    WHERE

    {

         {?s ?p ?o .
         ?p rdfs:subPropertyOf dm:HierarchicalRelations.
         ?s rdfs:label ?sl .
         ?o rdfs:label ?ol .    
         }

         }

     """

    get_s_nodes="""
    SELECT ?s ?q ?l
    WHERE
        {
        ?s ?p ?o.
        ?o rdfs:subClassOf dm:Definition.
        BIND ( rdf:type as ?p )
        BIND ( dm:UniqueIdentifier as ?q)
        ?s rdfs:label ?l
        }

    """

    ng = nx.DiGraph()
    c_tree = list(graph.query(row_hierarchy_sparql, initNs=namespace_d))
    print(c_tree)
    for e in [(e[0].value, e[1].value)  for e in c_tree ]:
        ng.add_edge(*e)

#    source = [n for n,d in  ng.out_degree() if d==0][0]

    unique_id_triples = []
    for uri, prop, lab in list(graph.query(get_s_nodes, initNs=namespace_d)):
        unique_id_triples.append((uri, prop, Literal(".".join(nx_walk_predecessors(ng,lab.value,invert=True,process=text_process)))))
    
    return unique_id_triples


In [ ]:
mappable_things_ext

In [ ]:
row_graph = gen_temp_entity_triples_from_row(mapping_data, mappable_things_ext, "http://temp", data_rows[0], row_id='row_1')




In [ ]:
visualize(row_graph)

In [ ]:
# connect additional context to row-graph
row_graph.parse ("datamodels_rdf.owl", format='xml')
dmns = rdflib.Namespace(onto.base_iri)
row_graph.bind('dm', dmns)
row_graph.bind('rdfs', RDFS)
namespace_d = {'dm': dmns, 'rdfs' : RDFS, 'rdf' : RDF}


In [ ]:
row_node_assignment_sparql = """SELECT distinct ?s ?p ?o 
WHERE 
{
    {?s dm:RowNodeParticipant ?o.
    BIND (dm:RowNodeParticipant as ?p)
    }
    UNION
    {?s dm:UniqueIdentifier ?o.
    BIND (dm:UniqueIdentifier as ?p)
    }
    UNION

    {?row dm:RowNodeParticipant ?s.
    ?s ?p ?o
    }

}
"""

qg = t2rdflibg(row_graph.query(row_node_assignment_sparql, initNs={'dm': dmns, 'rdfs' : RDFS, 'rdf' : RDF}))

visualize(qg)

In [ ]:
unique_id_triples=gen_unique_identity_triples(row_graph, namespace_d)
for t in unique_id_triples:
    row_graph.add(t)

In [ ]:
q="""SELECT distinct ?s ?p ?o
WHERE
    {?s ?p ?o .
     ?p rdfs:subPropertyOf dm:HierarchicalRelations.}
 
 """


q="""SELECT distinct ?s ?o
WHERE

{
    {?s ?p ?o .
     ?h rdfs:subPropertyOf dm:HierarchicalRelations.
     BIND (dm:UniqueIdentifier as ?p)
     ?s dm:UniqueIdentifier ?o .
     }
     
}
 
 """

for t in row_graph.query(q, initNs={'dm': dmns, 'rdfs' : RDFS, 'rdf' : RDF}):
    print(t)
    print()

qg = row_graph.query(q, initNs={'dm': dmns, 'rdfs' : RDFS, 'rdf' : RDF})
list(qg)

In [ ]:
#{dir(k):v for k,v in list(row_graph.query(q, initNs={'dm': dmns, 'rdfs' : RDFS, 'rdf' : RDF}))}

{v.value:k.n3()[1:-1] for k,v in list(row_graph.query(q, initNs={'dm': dmns, 'rdfs' : RDFS, 'rdf' : RDF}))}

In [ ]:
q="""SELECT distinct ?s ?p ?o
WHERE
    {?s ?p ?o .
     ?p rdfs:subPropertyOf dm:HierarchicalRelations.}
 
 """


q="""SELECT distinct ?s ?p ?o
WHERE

{
    {?s ?p ?o .
     ?p rdfs:subPropertyOf dm:HierarchicalRelations.}
     UNION
    {?s ?h ?q .
     ?h rdfs:subPropertyOf dm:HierarchicalRelations.
     BIND (rdfs:label as ?p)
    ?s rdfs:label ?o}
     UNION
    {?q ?h ?s .
     ?h rdfs:subPropertyOf dm:HierarchicalRelations.
     BIND (rdfs:label as ?p)
    ?s rdfs:label ?o}
     UNION
    {?s ?h ?q .
     ?h rdfs:subPropertyOf dm:HierarchicalRelations.
     BIND (rdf:type as ?p)
     ?s rdf:type ?o .
     }
     UNION
    {?a ?h ?b .
     ?h rdfs:subPropertyOf dm:HierarchicalRelations.
     ?a rdf:type ?s .
     ?s rdfs:subClassOf ?o
     BIND (rdfs:subClassOf as ?p)
     }
     UNION
    {?q ?h ?s .
     ?h rdfs:subPropertyOf dm:HierarchicalRelations.
     BIND (dm:UniqueIdentifier as ?p)
    ?s dm:UniqueIdentifier ?o}
     UNION
    {?s ?h ?q .
     ?h rdfs:subPropertyOf dm:HierarchicalRelations.
     BIND (dm:UniqueIdentifier as ?p)
     ?s dm:UniqueIdentifier ?o .
     }
     
}
 
 """

for t in row_graph.query(q, initNs={'dm': dmns, 'rdfs' : RDFS, 'rdf' : RDF}):
    print(t)
    print()

qg = t2rdflibg(row_graph.query(q, initNs={'dm': dmns, 'rdfs' : RDFS, 'rdf' : RDF}))

visualize(qg)

In [ ]:
def search_on_key_value(graph, key, search):
    search= str(search)
    sparql_base = """SELECT distinct ?s
    WHERE
        {
        ?s %%key%% ?q FILTER ( ?q = str("%%search%%") ) .
        }""".replace("%%search%%",search).replace("%%key%%", key)

    matches = graph.query(sparql_base, initNs={'dm': dmns, 'rdfs' : RDFS, 'rdf' : RDF})
    return matches


def get_row_objects(graph, row_id):
    

list(get_obj_rels(row_graph, "rdfs:label", "Core"))



In [ ]:
search= 'row_1'
sparql_base = """SELECT ?s ?p ?o
WHERE
    {
    {
    ?s dm:RowNodeParticipant ?o.
    ?s dm:UniqueIdentifier ?q FILTER ( ?q = str("search") ) .
    BIND ( dm:RowNodeParticipant as ?p)
    }
    UNION
    {
    ?s rdfs:label ?o.
    ?h dm:UniqueIdentifier ?q FILTER ( ?q = str("search") ) .
    ?h dm:RowNodeParticipant ?s.
    BIND ( rdfs:label as ?p)
    }
    UNION
    {
    ?s dm:UniqueIdentifier ?o.
    ?h dm:UniqueIdentifier ?q FILTER ( ?q = str("search") ) .
    ?h dm:RowNodeParticipant ?s.
    BIND ( dm:UniqueIdentifier as ?p)
    }
    }
    """.replace("search",search)

print(sparql_base)
print(list(row_graph.query(sparql_base, initNs={'dm': dmns, 'rdfs' : RDFS, 'rdf' : RDF})))
qg = t2rdflibg(row_graph.query(sparql_base, initNs={'dm': dmns, 'rdfs' : RDFS, 'rdf' : RDF}))

visualize(qg)

In [ ]:
list(onto.individuals())

In [ ]:
# Instantiate Empty graph fro this (and only this) data row:


In [ ]:
# Get all mappings:

dict(get_key_values(row_g, invert=True))

In [ ]:
del row_g

In [ ]:
def create_thing_triples(thing_label, owl_class, namespace, graph=None, pkey = None, test_for_existing=True):
    if graph is None:
        graph = rdflib.Graph()
    if test_for_existing:
        matches = [s for s,l in get_key_values(graph, pkey) if thing_label in l]
    if len(matches)>0:
        key = matches[0]
    else:
        key = uuid.uuid4().hex
    thing_URI = URIRef(key, namespace)
    # RDF:type, RDFS:label
    triples=[]
    props = { RDF.type : URIRef(owl_class.iri), 
              RDFS.label : Literal(thing_label)}
    for k,v in props.items():
        triples.append((thing_URI, k, v))
    return triples
    
def create_subject_spoes(ontology, subject, owl_property, obj_raw, graph=None, pkey = None, test_for_existing=True):
    namespace = ontology.base_iri
    if graph is None:
        graph = rdflib.Graph()
    subj_uri = URIRef(subject.iri)
    prop_uri = URIRef(owl_property.iri)
    if owl_property.is_a[0].iri in ['http://www.w3.org/2002/07/owl#DatatypeProperty']:
        obj = Literal(obj_raw)
    else:
        print(owl_property.is_a[0])
        obj = URIRef(obj_raw.iri)
    spo = (subj_uri, prop_uri, obj)
    return spo
    

In [ ]:
def instantiate_thing(label, tclass, space, test_for_existing=True):
    # Check to see if this object of the same class contains the same label
    # if so, then return the first item of that name
    if test_for_existing:
        matches = [i for i in space.search(type=tclass) if label in i.label]
        if len(matches)>0:
            return matches[0]
    # Otherwise, generate a unique ID, create the item in the space and assign this label
    guid = uuid.uuid4().hex
    with space:
        i = tclass(guid, namespace=space)
    i.label=label
    return i
    

In [ ]:

def get_thing_from_label_graphsearch(graph, domain, search_key=None):
    #sd_list = [c for c in list(space.classes()) if c==domain]
    
    sd_list = [s for s,p,o in graph.triples((None, RDF.type, URIRef(domain.iri)))]
    
    # Get instances of classes that match provided keys
    if search_key is None:
        c_s = [s for l in sd_list for s in l.instances()]
    else:
        c_s = [s for l in sd_list for s,p,o in graph.triples((None, RDFS.label, Literal(search_key)))]
    
    return c_s

def get_thing_from_label(space, domain, search_key=None):
    sd_list = [c for c in list(space.classes()) if c==domain]
    
    # Get instances of classes that match provided keys
    if search_key is None:
        c_s = [s for l in sd_list for s in l.instances()]
    else:
        c_s = [s for l in sd_list for s in l.instances() if search_key in s.label]
    
    return c_s
    

def instantiate_property(subject_key, subject_domain, predicate, object_key, object_domain, space):
    # Get classes for domains and ranges
    sd_list = [c for c in list(space.classes()) if c==subject_domain]
    od_list = [c for c in list(space.classes()) if c==object_domain]
    
    # Get instances of classes that match provided keys
    if subject_key is None:
        c_s = [s for l in sd_list for s in l.instances()]
    else:
        c_s = [s for s in sd_list for s in l.instances() if subject_key in s.label]
        
    if object_key is None:
        c_o = [o for l in od_list for o in l.instances()]
    else:
        print ("labels", [(o.label, object_key in o.label, object_key) for o in od_list])
        c_o = [o for l in od_list for o in l.instances()if object_key in o.label]
    
    print(od_list, object_key)
    
    print(predicate)
    predicate.python_name="pp"
    print(c_s[0], predicate)
    if len(c_o)==0:
        print("property_data", predicate, object_key, od_list, predicate.is_a)
        # Sometimes fails for data_properties
        # prop_relation = c_s[0].pp.append(object_key)
        #prop_relation = space._add_obj_triple_spo(c_s[0].storid, predicate.storid, object_key)
        try:
            prop_relation=predicate[c_s[0]]=[object_key]
        except AttributeError as e:
            print(e)
            prop_relation=predicate[c_s[0]]=[object_key]
        print("success", prop_relation)
        print("value", predicate[c_s[0]], c_s[0])
    else:
        if predicate.is_functional_for(c_o[0]):
            prop_relation = c_s[0].pp=c_o[0]
        else:
            prop_relation = c_s[0].pp=[c_o[0]]
    predicate.python_name = ""
    return c_s, predicate, c_o, prop_relation

In [ ]:
#ont = cleandms
#ont = onto
#q = instantiate_thing("test", onto.Class, cleandms, True)
#q = instantiate_thing("test 2", onto.Class, onto, True)

#for i in ont.Class.instances():
#    print (i.iri, i.label)

In [ ]:
#for i in onto.Class.instances():
#    print (i.iri, i.label)

In [ ]:
s_objects = []
p_objects = []
for s in serializations:
    print(s)
    mappings = [m for m in s.SerializationContainsMapping]
    serials = {}
    for m in mappings:
        s_key = m.SerializationLabel.first()
        i_key = m.MappingLabel.first()

        if s_key not in serials.keys():
            serials[s_key]=[]

        print ("\t", m, s_key)
            
        if m.is_a.first() == onto.EntityMapping:
            print("\t\tEntity:{e}".format(e=m))
            o_range = m.MappingRange.first()
            property_d = { "type" : "entity", 
                           "mapping_name" : i_key, 
                           "property" : "rdf:type:", 
                           "range" :    mappable_things[o_range], 
                           "property_type" : mappable_things_ext[i_key],
                           "subject_label" : data_row[s_key]}
#            serials[s_key].append((data_row[s_key], "rdf:type", mappable_things[o_range]))
            serials[s_key].append(property_d)
    
        elif m.is_a.first() == onto.PropertyMapping:
            print("\t\tProperty:{e}".format(e=m))
            s_domain = m.MappingDomain.first()
            o_range = m.MappingRange.first()
            property_d = { "type" : "property", 
                           "mapping_name" : i_key, 
                           "domain" :   mappable_things[s_domain], 
                           "property" : mappable_things[i_key], 
                           "property_type" : mappable_things_ext[i_key],
                           "range" :    mappable_things.get(o_range,o_range), 
                           "range_search_value" : data_row[s_key]}
            serials[s_key].append(property_d)
            
        else:
            print(m.is_a)
            print(s_key)
            assert False
            
print()

# Instantiate temporary versions of all the in-row objects
for v in serials.values():
    for u in v:
        print(u)
        if u['type'] == "entity":#u[1]=="rdf:type":
            #with onto:
                #q = instantiate_thing("test", onto.Class, cleandms, True)
            #s_objects.append(instantiate_thing(u['subject_label'], u['range'], onto, True))
            s_objects.append(create_thing_triples(u['subject_label'], u['range'], onto.iri, graph=None, pkey=RDFS.label, test_for_existing=True))
                #s_objects.append(instantiate_thing(u[0], u[2], onto, True))
                #u[2](u[0], namespace=onto) # Create instances in the ontology
            print(u)

print()


In [ ]:
# Instantiate Empty graph fro this (and only this) data row:
row_g = rdflib.Graph()

for obj_def in s_objects:
    for triple in obj_def:
        print(triple)
        row_g.add(triple)

In [ ]:

# Create property links for all identified in-row objects
for v in serials.values():
    for u in v:
        if u['type'] == "property":

            p_objects.append(u)
            # instantiate_property(None, u['domain'], u['property'], u["range_search_value"], u['range'], onto)
            subject = get_thing_from_label(onto, u['domain'], search_key=None)
            print(u['domain'], subject)
            cand_obj = get_thing_from_label(onto, u['range'], search_key=u["range_search_value"])
            create_subject_spoes(onto, subject, u['property'], cand_obj, graph=None, pkey = None, test_for_existing=True)


In [ ]:
g = onto.world.as_rdflib_graph()
for s,p,o in g.triples((URIRef("http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#9cd507c514af420ea52a9e7fa46f06f4"), None, None)):
    print(s,p,o)

In [ ]:
for s,p,o in g.triples((None, RDF.type, None)):
    print(s,p,o)

In [ ]:
am = list(onto.individuals())[0]

In [ ]:
assert False

In [ ]:
dm=[i for i in onto.individuals() if 'test' in i.label][0]
#dm.get_properties()
print (dm, dm.is_a)
for p in dm.INDIRECT_get_properties():
    print(p,p[dm])

In [ ]:
p_objects

In [ ]:
p_objects[-2]

In [ ]:
for p in p_objects:
    print(p['property'][attrib])#dm)

In [ ]:
assert False

In [ ]:
onto.ModelTypeCode._python_name

In [ ]:
onto.ModelTypeCode.is_a[0].iri=="http://www.w3.org/2002/07/owl#DatatypeProperty"

In [ ]:
dir(onto.ModelTypeCode)

In [ ]:
[(i, i.is_a, i.label) for i in list(onto.individuals())]

In [ ]:
attrib=[i for i in onto.individuals() if 'PersonID' in i.label][0]
attrib.get_properties()


In [ ]:
for p in attrib.INDIRECT_get_properties():
    print(p,p[attrib])

In [ ]:
dm=[i for i in onto.individuals() if 'Principal Interactions' in i.label][0]
#dm.get_properties()
print (dm, dm.is_a)
for p in dm.INDIRECT_get_properties():
    print(p,p[dm])

In [ ]:
for e,p in enumerate(onto.data_properties()):
    print(p[dm])

In [ ]:
assert False

In [ ]:
for e,p in enumerate(onto.data_properties()):
    p[dm]=["Test"]
    print(e,p==[p for p in list(onto.data_properties()) ][8])
print("Done")
[p for p in list(onto.data_properties()) ][8][dm]=["Tester"]

In [ ]:
dir(onto)

In [ ]:
onto.Class.instances(), onto.Attribute.instances(), onto.DataModel.instances(), onto.ModelDomain.instances()

In [ ]:
onto.Attribute.instances()[0].get_properties()

In [ ]:
list(onto.Attribute.instances()[0]._get_instance_possible_relations())

In [ ]:
dir(onto.DataModel.instances()[0])

In [ ]:
for p in onto.Attribute.instances()[0].get_properties():
    print(p, p.python_name, p[onto.DataModel.instances()[0]])
        

In [ ]:
for p in onto.DataModel.instances()[0].get_properties():
    print(p, p.python_name, p[onto.DataModel.instances()[0]])
        

In [ ]:
onto.ModelTypeCode[onto.DataModel.instances()[0]]=['Test']

In [ ]:
onto.ModelTypeCode.python_name = "mtc"
#onto.DataModel.instances()[0].mtc=["Conceptual"]

In [ ]:
onto.DataModel.instances()[0].ModelTypeCode

In [ ]:
with onto:
    owlr.sync_reasoner(raw_world)

In [ ]:
[c for c in list(onto.classes()) if c==p_objects[0]['domain']][0].instances()

In [ ]:
p_objects[0]['property'].is_functional_for(p_objects[0]['range'])

In [ ]:
instantiate_property(None, p_objects[0]['domain'], p_objects[0]['property'], p_objects[0]['range_search_value'], p_objects[0]['range'], onto)

In [ ]:
[(s,s.is_a, s.label) for s in s_objects]

In [ ]:
[(c,c.iri, c.is_a, c.label) for c in onto.Class.instances()]

In [ ]:
assert False

In [ ]:
serials['Entity']

In [ ]:
with onto:
    owlr.sync_reasoner()

In [ ]:
onto.get_properties()

In [ ]:
list(onto.inconsistent_classes())

In [ ]:
for c in list(onto.classes()):
    print (c, c.instances())